In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd

from scipy.spatial import Delaunay

import anndata2ri
import logging
from scipy.sparse import issparse
from CSCORE.CSCORE_IRLS import CSCORE_IRLS

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro


rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
adata = sc.read_h5ad("spatial_brain_rongprocessed.h5ad")

adata.obs['n_counts'] = np.array(np.sum(adata.X, axis = 1))

if issparse(adata.X):
    if not adata.X.has_sorted_indices:
        adata.X.sort_indices()
ro.globalenv["adata"] = adata

variable_gene = pd.read_csv("rankgenelist_brain.csv", index_col=0)

ro.globalenv["select_gene"] = variable_gene.iloc[0:1000,:]


In [ ]:
%%R
seurat_obj = as.Seurat(adata, counts="X", data = NULL)
seurat_obj = RenameAssays(seurat_obj, originalexp = "RNA")
res = SCTransform(object=seurat_obj, vst.flavor = "v2", residual.features=rownames(select_gene) , method = "glmGamPoi", verbose = FALSE)

In [ ]:
var_cand = list(ro.r("seurat_obj@assays$RNA@counts@Dimnames[[1]]"))
adata.var_names = var_cand
gene_list = list(ro.r("rownames(res@assays$SCT@scale.data)"))
norm_x = ro.r("res@assays$SCT@scale.data")
exp_matrix = pd.DataFrame(norm_x, index=gene_list)


adata_new = adata[:,gene_list]

df = pd.DataFrame(adata_new.X.todense(), index=adata_new.obs_names, columns = adata_new.var_names)

df.to_csv("heart_atlas/visium_spatial_data.csv")

counts = adata_new.X
seq_depth = adata_new.obs['n_counts'].values

B_cell_result = CSCORE_IRLS(np.array(counts), seq_depth)
p_value = B_cell_result[1]
cor_matrix = (p_value<0.005)*1
print(cor_matrix)

exp_matrix.to_csv(f"./brain_atlas/spatial_brain_rongse"+"_rna_expression.csv")
cor_matrix = pd.DataFrame(cor_matrix, index = gene_list, columns = gene_list)
cor_matrix.to_csv(f"./brain_atlas/spatial_brain_rongse"+"_pvalue.csv")